In [260]:
import pandas as pd
# Input could be like: Monday 8:00am - 10:00am; Wednesday 12:00pm - 2:00pm; 10 weeks
    
# weight the importance of filling a timeslot if someone is available for it by the proportion of people available
    # if, 5/5 people are available for 4 timeslots but only 1 person is available for the 5th
        # start by getting inverse proportion at each timelot, then divide by sum 
            # if no one is availble then set this to 0 and remove from consideration
# weight the cost of a person filling a timeslot by the number of timeslots they've already been assigned to 
    # and the num of timeslots the person is available 
        # num timeslots unfilled / num. available for the person
        
# combine by each person getting a: and taking person with maximum 
# | sum(current timeslot weights available for the person) / (sum(current unfilled timeslot weights) ) 
#   - sum(timeslot weights available for the person with filling) / (sum(unfilled timeslot weights with filling)))|
#   / (number_tours_filled_for_person / avg._tours_filled)
# this prioritizes using the person to fill a slot that has the most unique availability 

def read_sheets(file):
    sheets = pd.read_excel(file, sheet_name=None, header=None, dtype='str',
                           names=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])
    non_sheets = set(['New Ambassador', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])
    
    # preprocess sheets
    names = set(sheets.keys()).difference(non_sheets)
    for name in names:
        cur_schedule = sheets[name].iloc[3:].reset_index()
        cur_schedule['index'] = cur_schedule['index'].str[:7]
        sheets[name] = cur_schedule.set_index('index')

    return sheets, names

def get_schedule_table(weeks, day_times, num_people_per_shift):
    ind = [[], []]
    for week in range(1, weeks + 1):
        for day in day_times:
            ind[0] += [day[0] + str(week) for _ in range(len(day) - 1)]
            ind[1] += day[1:]

    return pd.DataFrame(index=ind, columns=['Name ' + str(i) for i in range(1, num_people_per_shift + 1)])

def get_timeslot_availabilities(sheets, names, day_times):
    availabilities = {}
    for day in day_times:
        for time in day[1:]:
            names_available = []
            for name in names:
                time_period = time.replace(" ", "").split("-")
                # returns true if all values are 1 for the time period (add functionality for '/' later)
                if (sheets[name].loc[time_period[0]:time_period[1]].iloc[:-1][day[0]] == '1').all():
                    names_available.append(name)
            availabilities[day[0] + time] = set(names_available)
    return availabilities

def get_timeslot_weights(availabilities, weeks, num_people_per_shift):
    weights = {i: len(names) / len(availabilities[i]) if len(availabilities[i]) != 0 else 0 for i in availabilities.keys()}
    
    total = sum(weights.values())
    for key in weights.keys():
        weights[key] /= (total * weeks * num_people_per_shift)
    return weights

# combine by each person getting a: and taking person with maximum 
# | sum(current timeslot weights available for the person) / (sum(current unfilled timeslot weights) ) 
#   - sum(timeslot weights available for the person with filling) / (sum(unfilled timeslot weights with filling)))|
#   / (number_tours_filled_for_person / avg._tours_filled)
# this prioritizes using the person to fill a slot that has the most unique availability 

# this assumes availabilities has been trimmed for each person on each iteration of filling a timeslot
def get_sum_timeslot_weights_for_all(names, availabilities, timeslot_weights, weeks):
    sum_timeslot_weights = {name: 0 for name in names}
    for name in names:
        for key in availabilities.keys():
            if name in availabilities[key]:
                sum_timeslot_weights[name] += timeslot_weights[key] * weeks
    
    return sum_timeslot_weights

def get_best_person_for_timeslot(timeslot, names, availabilities, timeslot_weights, timeslot_weight_sums,
                                 tours_per_person, avg_tours_assigned, timeslot_weight_accounted): 
    people_ratings = {}
    if len(availabilities[timeslot]) > 0:
        for name in availabilities[timeslot]:
            tot_timeslot_weight_unaccounted = 1 - timeslot_weight_accounted
            first = timeslot_weight_sums[name] / tot_timeslot_weight_unaccounted
            to_be_accounted = timeslot_weights[timeslot]
            second = (timeslot_weight_sums[name] - to_be_accounted) / (tot_timeslot_weight_unaccounted - to_be_accounted)
            people_ratings[name] = abs(first - second) / ((tours_per_person[name] + 1) / (avg_tours_assigned + 1))
        return sorted(people_ratings, key=people_ratings.get, reverse=True)[0]
    return None
    
def schedule_people(schedule_table, names, availabilities, timeslot_weights, weeks):
    # initialize cumulative proportion of timeslot weights for each person's availability
    timeslot_weight_sums = get_sum_timeslot_weights_for_all(names, availabilities, timeslot_weights, weeks)
    num_tours_assigned = 0
    tours_per_person = {name: 0  for name in names}
    timeslot_weight_accounted = 0
    for day_time in schedule_table.index:
        removed = []
        for col in schedule_table.columns:
            timeslot = day_time[0][:-1] + day_time[1]
            avg_tours_assigned = num_tours_assigned / len(names)
            assignment = get_best_person_for_timeslot(timeslot, names, availabilities, timeslot_weights,
                                                      timeslot_weight_sums, tours_per_person, avg_tours_assigned,
                                                      timeslot_weight_accounted)
            if assignment is not None:
                # reduce cumulative proportion of timeslot weights
                for name in availabilities[timeslot]:
                    timeslot_weight_sums[name] -= timeslot_weights[timeslot]
                timeslot_weight_accounted += timeslot_weights[timeslot]
                num_tours_assigned += 1
                tours_per_person[assignment] += 1
                schedule_table.loc[day_time, col] = assignment
                # if multiple assignments for a particular day_time then need to remove from consideration
                availabilities[timeslot].remove(assignment)
                removed.append(assignment)
        availabilities[timeslot].update(removed)

    return schedule_table, tours_per_person

file = 'tour-availability.xlsx'
sheets, names = read_sheets(file)
weeks = 2
num_people_per_shift = 2
# currently requires this input format
day_times = [['Monday', '07:30am - 08:00am', '12:00pm - 02:00pm'], ['Wednesday', '12:00pm - 02:00pm']]
schedule_table = get_schedule_table(weeks, day_times, num_people_per_shift)
availabilities = get_timeslot_availabilities(sheets, names, day_times)
timeslot_weights = get_timeslot_weights(availabilities, weeks, num_people_per_shift)

ret = schedule_people(schedule_table, names, availabilities, timeslot_weights, weeks)
display(ret[0])
ret[1]

Name 1        Name 2
Monday1    07:30am - 08:00am  Charlie Camilli   Alex Kuesel
           12:00pm - 02:00pm       Ben Gibson  Charlie Ward
Wednesday1 12:00pm - 02:00pm      Alex Kuesel  Charlie Ward
Monday2    07:30am - 08:00am  Charlie Camilli    Ben Gibson
           12:00pm - 02:00pm     Charlie Ward   Alex Kuesel
Wednesday2 12:00pm - 02:00pm       Ben Gibson   Alex Kuesel

{'Ben Gibson': 3,
 'Sean Wahl': 0,
 'Saif Younis': 0,
 'Charlie Ward': 3,
 'Melina Ruano': 0,
 'Alex Schwartz': 0,
 'Alex Kuesel': 4,
 'Charlie Camilli': 2}